In [ ]:
!pip install jellyfish

In [ ]:
from datasets import DatasetDict, Dataset

## FLAN2021

In [ ]:
from datasets import load_dataset
flan2021 = load_dataset("conceptofmind/flan2021_submix_original", split="train", streaming=True)

- Summarize the given document.
- Document: {tok_1 ... tok_n}
- Summary: {tok_1 ... tok_m}

In [ ]:
# summarize : huggingface:xsum, gigaword:1.2.0, multi_news:1.0.0, gem/wiki_lingua_english_en:1.1.0
L = []
for example in flan2021 :
    if "fs" in example["template_type"] :
        continue
    if "summarize " in example["inputs"] :
        L.append(example)

In [ ]:
start_blacklist = [
    "Please briefly summarize this news article:",
    "Briefly summarize this sentence:"
]
end_blacklist = [
    "Summary:",
    "Briefly summarize that dialogue."
]
blacklist = [
    "What best summarizes the content of the above article?"
]
count = 0
prompts = []
W = []
instruction = "Summarize the given document."
for example in L :
    prompt = example["inputs"]
    completion = example["targets"]
    found = False
    for k in start_blacklist :
        if prompt.startswith(k) :
            prompt = prompt[len(k):].strip()
            found = True
    if not found :
        continue
    for k in end_blacklist :
        if prompt.endswith(k) :
            prompt = prompt[:-len(k)].strip()
    skip = False
    for k in blacklist :
        if prompt.find(k) != -1 :
            skip = True
            break
    if skip : 
        continue
    if prompt in prompts :
        continue
    else :
        prompts.append(prompt)
    count +=1
    out = f"{instruction}\n"+"Document: {"+prompt+"}\nSummary: {"+completion+"}"
    W.append(out)
    print(out)
    print("-"*200)

In [ ]:
count == len(W)

In [ ]:
summary_dataset = Dataset.from_dict(
    {
        "prompt" : [e for e in W]
    }
)

In [ ]:
summary_dataset.save_to_disk("summary_dataset")

In [ ]:
cnndm = flan2021.filter(lambda x : x["task_name"] == 'cnn_dailymail:3.4.0')

In [ ]:
start_blacklist = [
    "Write some highlights for the following article:",
    "Write highlights for this article:"
]
end_blacklist = [
    "What are the important parts of this article?",
    "What are highlight points for this article?",
    "Here is a summary of the highlights for this article:",
    "Write highlights for this article.",
    "Summarize the highlights of this article.",
    "Highlights:",
]
start_reverse = [
    "Write an article using the following points:",
    "Use the following highlights to write an article:"
]
end_reverse = [
    "Write an article based on these highlights.",
]
K = []
for i, example in enumerate(cnndm):
  """
  if i < 1000 :
    pass
  if i >= 2000 :
    break
  """
  if "zs" not in example["template_type"] :
    continue
  prompt = example["inputs"]
  completion = example["targets"]
  reverse = False
  for k in start_reverse :
    if prompt.startswith(k):
      reverse = True
      prompt = prompt[len(k):]
      prompt, completion = completion, prompt
      break
  for k in end_reverse :
    if prompt.endswith(k):
      reverse = True
      prompt = prompt[:-len(k)]
      prompt, completion = completion, prompt
      break
  for k in start_blacklist :
    if prompt.startswith(k) :
      prompt = prompt[len(k):]
      break
  for k in end_blacklist :
    if prompt.endswith(k) :
      prompt = prompt[:-len(k)]
  prompt = prompt.strip()
  completion = completion.strip()
  for k in ["Article:"]:
    if completion.endswith(k):
      completion = completion[:-len(k)].strip()
      break
  K.append((prompt, completion))
  out = f"INPUT\n{prompt}\n\nOUTPUT\n{completion}"
  print(out)
  print("-"*100)

## Dialog

In [ ]:
from datasets import load_dataset
dialog = load_dataset("conceptofmind/dialog_submix_original", split="train")

In [ ]:
for i, example in enumerate(dialog) :
    if i >= 1000 :
        break
    if "zs" not in example["template_type"] :
        continue
    prompt = example["inputs"]
    completion = example["targets"]
    print(f"Question:\n{prompt}\n\nAnswer:\n{completion}")
    print("-"*100)

## T0

In [ ]:
from datasets import load_dataset
t0 = load_dataset("conceptofmind/t0_submix_original", split="train")

In [ ]:
tasks = {}
for task in t0["task_name"] :
    if task in tasks :
        tasks[task] += 1
    else :
        tasks[task] = 1
print(tasks)
template_types = {}
for template_type in t0["template_type"] :
    if template_type in template_types :
        template_types[template_type] += 1
    else :
        template_types[template_type] = 1
print(template_types)

In [ ]:
#'wiki_qa_automatic_system' : yes/no
for i, example in enumerate(t0):
    if i >= 100000 :
        break
    if "zs" not in example["template_type"] :
        continue
    prompt = example["inputs"]
    completion = example["targets"]
    if "Options:" in prompt :
        print(f"INPUT\n{prompt}\n\nOUTPUT\n{completion}")
        print("-"*100)

In [ ]:
K = []

In [ ]:
start_blacklist = ["Subject:", "Question:", "Answer the following question:", "Q:", "Given the question:", 
    "Please answer the following question:", "Read the article and select the best answer.",
    "I'm taking a test and have to guess the right answer to the question after the article.",
    "Read the article and select the best answer.", "Read the following context and choose the correct option to answer the question.",
    "Pick the option in line with common sense to answer the question.",
    "I'm taking a test and have to guess the right answer to the question after the article."
]
end_blacklist = ["A:", "Answer:", "The answer is:", "The answer to this question is:",
                "=== The correct answer is"]
for i, example in enumerate(t0) :
    if i >= 10000000 :
        break
    if "zs" not in example["template_type"] :
        continue
    prompt = example["inputs"]
    completion = example["targets"]
    if len(completion) == 1 :
        for element in start_blacklist:
            if prompt.strip().startswith(element) :
                #print(f"BEFORE\n{prompt}\n\nAFTER\n{prompt.strip()[len(element):].strip()}")
                prompt = prompt.strip()[len(element):].strip()
                break
        #print("-"*200)
        if "Options:" in prompt :
            garbage = "=== The correct answer is"
            end = prompt.find("Options:")
            header = prompt[:end]
            remainder = prompt[end + len("Options: "):]
            candidates = ["A. ", "B. ", "C. ", "D. "]
            last = "E. "
            if min([remainder.find(c) for c in candidates]) >= 0 :
                pass
            else :
                candidates = ["A: ", "B: ", "C: ", "D: "]
                last = "E: "
            clean = header.strip()+"\n"
            more = None
            for j, element in enumerate(candidates):
                if j == 3 :
                    if remainder.find(last) == -1 :
                        option = remainder[remainder.find(element)+len(element):]
                    else :
                        option = remainder[remainder.find(element)+len(element):remainder.find(last)]
                        more = remainder[remainder.find(last)+len(last):]
                else :
                    option = remainder[remainder.find(element)+len(element):remainder.find(candidates[j+1])]
                #print(f"OPTION\n{option}")
                clean += f"{element}{option}\n"
                if more :
                    clean += f"{last}{more}\n"
            prompt = clean.strip()
            for element in ["Context: ", "Article: ", "Question: "] :
                index = prompt.find(element)
                if index != -1 :
                    left = prompt[0:index]
                    right = prompt[index+len(element):]
                    prompt = left+"\n"+right
                    break
            for element in end_blacklist :
                if prompt.strip().endswith(element) :
                    prompt = prompt.strip()[:-len(element)].strip()
                    break
            # SECOND CHECKING
            for element in start_blacklist:
                if prompt.strip().startswith(element):
                    prompt = prompt.strip()[len(element):].strip()
                    break
            for element in end_blacklist :
                if prompt.strip().endswith(element) :
                    prompt = prompt.strip()[:-len(element)].strip()
                    break
            #print(f"INPUT\n{prompt}\n\nOUTPUT\n{completion}\n\nGATE\n{example['inputs']}")
            #print(f"INPUT\n{prompt}\n\nOUTPUT\n{completion}")
            #print("-"*200)
            K.append((prompt, completion))
        else :
            continue
        #print(f"INPUT\n{prompt}\n\nOUTPUT\n{completion}")
        #print("-"*200)

In [ ]:
import jellyfish
threshold = 0.8

In [ ]:
prompts = []
W = []
for i, (a, b) in enumerate(K) :
    duplicate = False
    for c in prompts :
        #if jellyfish.jaro_similarity(a, c) >= threshold :
        if c == a :
            duplicate = True
            break
    if duplicate :
        continue
    prompts.append(a)
    a = a.replace("A: ", "A. ")
    a = a.replace("B: ", "B. ")
    a = a.replace("C: ", "C. ")
    a = a.replace("D: ", "D. ")
    a = a.replace("E: ", "E. ")
    out = f"Question: {a.strip()}\nAnswer: {b.strip()}"
    W.append(out)
    print(out)
    print("-"*200)

In [ ]:
len(W)

In [ ]:
qcm_dataset = Dataset.from_dict(
    {
        "prompt" : [e for e in W]
    }
)

In [ ]:
qcm_dataset.save_to_disk("flan2021_qcm_dataset")

## CoT

In [ ]:
mixture=[
    ('flan2021_submix', 0.4),  # mixing weight = 40%
    ('t0_submix', 0.32),       # mixing weight = 32%
    ('niv2_submix', 0.2),      # mixing weight = 20%
    ('cot_submix', 0.05),      # mixing weight = 5%
    ('dialog_submix', 0.03),   # mixing weight = 3%
]
templates = {
    "next question:" : "reasoning",
    "[Q]" : "[A]",
    #"question:" : "answer:",
    "Question:" : "Answer:",
    "[Question]:" : "[Answer]:",
    "Q:": "A:",
    "question in book:": "standard solution:",
    "[TEACHER]": "[Student]",
    "QUESTION:" : "ME:",
    "q:" : "a:",
    "Jax:" : "Alex:",
    "QUESTION:" : "SOLUTION:",
    "Student:" : "Teacher:",
    "[1]" : '[2]',
    "Leo:": "Mei:",
    "Answer the following question." : "The answer is",
    "Step-by-step reasoning process:" : "[Q & A]"
}

In [ ]:
from datasets import load_dataset
ds = load_dataset("conceptofmind/cot_submix_original", split="train")

In [ ]:
tasks = {}
for task in ds["task_name"] :
    if task in tasks :
        tasks[task] += 1
    else :
        tasks[task] = 1
print(tasks)
template_types = {}
for template_type in ds["template_type"] :
    if template_type in template_types :
        template_types[template_type] += 1
    else :
        template_types[template_type] = 1
print(template_types)

In [ ]:
ds

In [ ]:
for example in ds :
    if "fs" in example["template_type"] and "[Question]" in example["inputs"]:
        print(f"{example['inputs']}\n\n{example['targets']}")
        break

### gsm8k

In [ ]:
start_blacklist = ["My question is:", "Question:",
        "Give the step-by-step reasoning process and then the final answer.",
        "Consider the question.", "Question."
]
end_blacklist = [
    "Chain of thought:", 
    "Let's answer step by step:",
    "Your thoughts:",
    "I think this is the detailed solution:",
    "Think carefully first, then make a decision:",
    "Thoughts? Step-by-step reasoning:",
    "What step-by-step reasoning justifies that answer?"
]
black_completion = [
    "The question is:",
    "Q:"
]
black_prompt = [
    "Given the following reasoning and answer, what was the question?",
    "For this chain-of-thought reasoning and answer, what was the question?",
    "Given the rationale, provide a reasonable question and answer.",
    "Reconstruct a question, answer pair from this explanation:"
]
K = []
for i, example in enumerate(ds):
    if i >= 183848 :
        break
    if (example["task_name"] != "cot_gsm8k" and example["task_name"] != "cot_gsm8k_ii" ):
        continue
    prompt = example['inputs']
    completion = example['targets']
    x_shot = example["template_type"]
    if x_shot == "zs_opt" :
        if prompt.startswith("Lizzy:") :
            prompt = prompt[len("Lizzy:"):]
            me = prompt.find("Me:")
            prompt = prompt[:me].strip()
        for k in end_blacklist :
            if prompt.strip().endswith(k):
                prompt = prompt.strip()[:-len(k)]
                break
        for k in start_blacklist :
            if prompt.startswith(k) :
                prompt = prompt[len(k):].strip()
                break
        skip = False
        for k in black_completion :
            if completion.startswith(k) :
                skip = True
                break
        for k in black_prompt :
            if prompt.startswith(k) :
                skip = True
                break
        if skip :
            continue
        K.append((prompt, completion))
        #print(f"INPUT\n{prompt}\n\nOUTPUT\n{completion}")
        #print("-"*200)
    else :
        pass
        """
        found = False
        for k, v in templates.items() :
            if prompt.strip().endswith(v) :
                index = prompt.strip().rfind(k)
                if index == -1 :
                    continue
                prompt = prompt[index+len(k):-len(v)].strip()
                found = True
                break
        if completion.startswith("Step-by-step reasoning process:") :
            completion = completion[len("Step-by-step reasoning process: "):]
        if not found :
            #print(f"INPUT\n{prompt}\n\nOUTPUT\n{completion}")
            #print("-"*200)
        """

In [ ]:
len(K)

In [ ]:
print(len(K))
prompts = []
W1 = []
for i, (a, b) in enumerate(K) :
    duplicate = False
    for c in prompts :
        #if jellyfish.jaro_similarity(a, c) >= threshold :
        if c == a :
            duplicate = True
            break
    if duplicate :
        continue
    prompts.append(a)
    out = f"Question: {a.strip()}\nAnswer: {b.strip()}"
    W1.append(out)
    print(out)
    print("-"*200)
print(len(W1))

In [ ]:
W1new = []
for a in W1 :
    found = False
    for k in ["Explanation:", "Reasoning:", "Q & A:", "this implicit rationale, and corresponding answer"
        "stream-of-consciousness", "reasoning", "the stream of consciousness", "rationale"
    ]:
        if a.find(k) != -1 :
            found = True
            break
    if found :
        continue
    a = a.replace("Final answer:", "The answer is")
    a = a.replace("The final answer:", "The answer is")
    a = a.replace("So the final answer is", "The answer is")
    a = a.replace("Thus, the answer is", "The answer is")
    a = a.replace("Therefore, the answer is", "The answer is")
    a = a.replace("The answer:", "The answer is")
    a = a.replace("So the answer is", "The answer is")
    i = a.find("The answer is")
    if i == -1 :
        continue
    if a[i-1] == "\n" :
        if a[i-2] == "." :
            a = a[:i-1]+" "+a[i:]
        else :
            a = a[:i-1]+a[i:]
    a = a.replace("Let's think first.", "")
    a = a.replace("Give me reasons, before answering the question", "")
    W1new.append(a)
    print(a)
    print("-"*100)
print(len(W1new))

In [ ]:
cot_gsm8k = Dataset.from_dict(
    {
        "prompt" : [e for e in W1new]
    }
)

### stream aqua

In [ ]:
start_blacklist = ["Q:", "Question:", "Some question:", "Student:"]
end_blacklist = ["Another student: Let's say, hmmm...",
                 "Think first, then make a decision. Some random thoughts:",
                 "Now, let's think a bit. Some random thoughts:",
                 "Let's think first. Stream of consciousness:",
                 "Let's give some random thoughts before answering.",
                 "Let's think first. Some random reasoning:",
                 "Random thoughts:",
                 "OK. Let's think. Some random thoughts first:",
                 "Hmmm, my stream of consciousness:",
                 "Stream of consciousness:",
                 "Stream of consciousness first, then make a decision:",
                 "Some stream of consciousness:"
                ]
K = []
for i, example in enumerate(ds) :
    if i >= 200000 :
        break
    #if example["task_name"] == "cot_sensemaking" : # sentence nonsensical
    #if example["task_name"] == "cot_esnli_ii" : # premise, hypothesis
    #if example["task_name"] == "cot_esnli" : # premise, hypothesis
    #if example["task_name"] == 'cot_ecqa' : # options
    #if example["task_name"] == "stream_qed" : # reading comprehension
    #if example["task_name"] == "cot_creak_ii" : # not
    #if example["task_name"] == "cot_strategyqa" : # general knowledge
    #if example["task_name"] == 'cot_qasc_ii' : # bete
    if example["task_name"] == 'stream_aqua' : # math
        if "zs" in example["template_type"] :
            prompt = example['inputs']
            completion = example['targets']
            for c in start_blacklist :
                if prompt.strip().startswith(c) :
                    prompt = prompt.strip()[len(c):]
                    break
            for c in end_blacklist :
                if prompt.strip().endswith(c):
                    prompt = prompt.strip()[:-len(c)]
                    break
            index = prompt.find("Options:")
            if index == -1 :
                continue
            left = prompt[:index]
            right = prompt[index+len("Options:"):]
            new_prompt = left.strip()+"\n"+right.strip()
            for (a, b) in [
                ("(A) ", "A. "),
                ("(B) ", "B. "),
                ("(C) ", "C. "),
                ("(D) ", "D. "),
                ("(E) ", "E. "),
            ] :
                new_prompt = new_prompt.replace(a, b)
                completion = completion.replace(a.strip(), b.strip()[:-1])
            K.append((new_prompt, completion))
            #print(f"INPUT\n{new_prompt}\n\nOUTPUT\n{completion}")
            #print("-"*200)

In [ ]:
print(len(K))
prompts = []
W2 = []
for i, (a, b) in enumerate(K) :
    duplicate = False
    for c in prompts :
        #if jellyfish.jaro_similarity(a, c) >= threshold :
        if c == a :
            duplicate = True
            break
    if duplicate :
        continue
    prompts.append(a)
    out = f"Question: {a.strip()}\nAnswer: {b.strip()}"
    out = out.replace("Final answer:", "The answer is")
    out = out.replace("The final answer:", "The answer is")
    out = out.replace("So the final answer is", "The answer is")
    out = out.replace("Thus, the answer is", "The answer is")
    out = out.replace("Therefore, the answer is", "The answer is")
    out = out.replace("The answer:", "The answer is")
    out = out.replace("So the answer is", "The answer is")
    out = out.replace("So the final answer", "The answer is")
    W2.append(out)
    print(out)
    print("-"*200)
print(len(W2))

In [ ]:
cot_aqua = Dataset.from_dict(
    {
        "prompt" : [e for e in W2]
    }
)

In [ ]:
cot_aqua[0]

### ecqa

In [ ]:
start_blacklist = [
    "Use reasoning to lead to the answer of the following question:",
    "I'll give you a question, please answer with step-by-step reasoning process.",
    "next question:"
]
end_blacklist = [
    "Step-by-step reasoning process below:",
    "Hmmm, let me think.",
    "Let's think now! Step-by-step reasoning:",
    "reasoning:",
    "Reasoning process:",
    "Let's give stream of consciousness first:",
    "Let's think step by step:",
    "Please answer and provide answer explanation.",
    "Let's think carefully first. Step-by-step reasoning process:"
]
K = []
for i, example in enumerate(ds) :
    if i >= 100000 :
        break
    #if example["task_name"] == "cot_sensemaking" : # sentence nonsensical
    #if example["task_name"] == "cot_esnli_ii" : # premise, hypothesis
    #if example["task_name"] == "cot_esnli" : # premise, hypothesis
    #if example["task_name"] == 'cot_ecqa' : # options
    #if example["task_name"] == "stream_qed" : # reading comprehension
    #if example["task_name"] == "cot_creak_ii" : # not
    #if example["task_name"] == "cot_strategyqa" : # general knowledge
    #if example["task_name"] == 'cot_qasc_ii' : # bete
    #if example["task_name"] == 'stream_aqua' : # math
    if example["task_name"] == 'cot_ecqa' : # options
        if "zs" in example["template_type"] :
            prompt = example['inputs']
            completion = example['targets']
            for c in start_blacklist :
                if prompt.strip().startswith(c) :
                    prompt = prompt.strip()[len(c):]
                    break
            for c in end_blacklist :
                if prompt.strip().endswith(c):
                    prompt = prompt.strip()[:-len(c)]
                    break
            index = prompt.find("Options:")
            if index == -1 :
                continue
            left = prompt[:index]
            right = prompt[index+len("Options:"):]
            new_prompt = left.strip()+"\n"+right.strip()
            for (a, b) in [
                ("(A) ", "A. "),
                ("(B) ", "B. "),
                ("(C) ", "C. "),
                ("(D) ", "D. "),
                ("(E) ", "E. "),
            ] :
                new_prompt = new_prompt.replace(a, b)
                completion = completion.replace(a.strip(), b.strip()[:-1])
            K.append((new_prompt, completion))
            #print(f"INPUT\n{new_prompt}\n\nOUTPUT\n{completion}")
            #print("-"*200)

In [ ]:
print(len(K))
prompts = []
W3 = []
for i, (a, b) in enumerate(K) :
    duplicate = False
    for c in prompts :
        #if jellyfish.jaro_similarity(a, c) >= threshold :
        if c == a :
            duplicate = True
            break
    if duplicate :
        continue
    prompts.append(a)
    out = f"Question: {a.strip()}\nAnswer: {b.strip()}"
    out = out.replace("Final answer:", "The answer is")
    out = out.replace("The final answer:", "The answer is")
    out = out.replace("So the final answer is", "The answer is")
    out = out.replace("Thus, the answer is", "The answer is")
    out = out.replace("Therefore, the answer is", "The answer is")
    out = out.replace("The answer:", "The answer is")
    out = out.replace("So the answer is", "The answer is")
    out = out.replace("So the final answer", "The answer is")
    W3.append(out)
    print(out)
    print("-"*200)
print(len(W3))

In [ ]:
cot_ecqa = Dataset.from_dict(
    {
        "prompt" : [e for e in W3]
    }
)

In [ ]:
cot_ecqa[0]

### strategyqa

In [ ]:
start_blacklist = [
    "Answer the following question by reasoning step-by-step.",
    "Answer the following question, but give the rationale first.",
    "Q:",
    "Let's answer this question slowly:"

]
end_blacklist = [
    "Given the above question, please answer with reasoning first!",
    "Think carefully first, then make a decision:",
    "Hmmm, my chain of thoughts:",
    "Think slowly and carefully, before giving your answer.",
    "Chain of thought:",
    "Now, let's think step by step:",
    "Please answer step by step:"

]
K = []
for i, example in enumerate(ds) :
    if i >= 100000 :
        break
    #if example["task_name"] == "cot_sensemaking" : # sentence nonsensical
    #if example["task_name"] == "cot_esnli_ii" : # premise, hypothesis
    #if example["task_name"] == "cot_esnli" : # premise, hypothesis
    #if example["task_name"] == 'cot_ecqa' : # options
    #if example["task_name"] == "stream_qed" : # reading comprehension
    #if example["task_name"] == "cot_creak_ii" : # not
    if example["task_name"] == "cot_strategyqa" : # general knowledge
    #if example["task_name"] == 'cot_qasc_ii' : # bete
        if "zs" in example["template_type"] :
            prompt = example['inputs']
            completion = example['targets']
            for c in start_blacklist :
                if prompt.strip().startswith(c) :
                    prompt = prompt.strip()[len(c):]
                    break
            for c in end_blacklist :
                if prompt.strip().endswith(c):
                    prompt = prompt.strip()[:-len(c)]
                    break
            K.append((prompt.strip(), completion.strip()))
            print(f"INPUT\n{prompt}\n\nOUTPUT\n{completion}")
            print("-"*200)

In [ ]:
print(len(K))
prompts = []
W4 = []
for i, (a, b) in enumerate(K) :
    duplicate = False
    for c in prompts :
        #if jellyfish.jaro_similarity(a, c) >= threshold :
        if c == a :
            duplicate = True
            break
    if duplicate :
        continue
    prompts.append(a)
    out = f"Question: {a.strip()}\nAnswer: {b.strip()}"
    out = out.replace("Therefore, the answer is", "The answer is")
    out = out.replace("Therefore, the final answer is", "The answer is")
    out = out.replace("The answer:", "The answer is")
    out = out.replace("Final answer:", "The answer is")
    out = out.replace("So, the final answer is", "The answer is")
    out = out.replace("So, the answer is", "The answer is")
    i = out.find("The answer is")
    if out[i-1] == "\n" :
        out = out[:i-1]+out[i:]
    W4.append(out)
    print(out)
    print("-"*200)
print(len(W4))

In [ ]:
cot_strategyqa = Dataset.from_dict(
    {
        "prompt" : [e for e in W4]
    }
)

In [ ]:
cot_strategyqa[0]

### stream_qed : reading comprehension

In [ ]:
start_blacklist = [
    "Quoc:", "Q:", "Give a stream of consciousness and then the final answer.",
    "Answer the following Q with stream of consciousness.",
    "I got a question for you:", "Output a stream of consciousness before answering the following.",
    "Use stream of consciousness to answer the following.",
    "Give some stream of consciousness and then the answer."
]
end_blacklist = [
    "HW Chung: OK, some thoughts:", "Let's think first:",
    "Okie... think carefully first, then make a decision:",
    "Let's think fast. Stream of consciousness:",
    "Let's give stream of consciousness below",
    "Steam of consciousness below:",
    "Let's give stream of consciousness first:",
    "Let's have some stream of consciousness first.",
    "OK. Let's think. My stream of consciousness:"
]
start_blacklist_completion = [
    "The relevant sentence in the passage is:",
    "To answer the above question, the relevant sentence is:",
    "This is the relevant information:",
    "The important part for answering this question is:",
    "To answer this question, we should know that:",
    "To answer the question, consider the following:",
    "The relevant information is:",
    "The relevant information to answer the above question is:"
]
K = []
for i, example in enumerate(ds) :
    if i >= 100000 :
        break
    #if example["task_name"] == "cot_sensemaking" : # sentence nonsensical
    #if example["task_name"] == "cot_esnli_ii" : # premise, hypothesis
    #if example["task_name"] == "cot_esnli" : # premise, hypothesis
    if example["task_name"] == "stream_qed" : # reading comprehension
    #if example["task_name"] == "cot_creak_ii" : # yes/no could require to reverse question and answer
    #if example["task_name"] == 'cot_qasc_ii' : # should reverse question and answer
        if "zs" in example["template_type"] :
            prompt = example['inputs']
            completion = example['targets']
            for c in start_blacklist :
                if prompt.strip().startswith(c) :
                    prompt = prompt.strip()[len(c):]
                    break
            for c in end_blacklist :
                if prompt.strip().endswith(c):
                    prompt = prompt.strip()[:-len(c)]
                    break
            for c in start_blacklist_completion :
                if completion.strip().startswith(c) :
                    completion = completion.strip()[len(c):]
                    break
            K.append((prompt.strip(), completion.strip()))
            print(f"INPUT\n{prompt.strip()}\n\nOUTPUT\n{completion.strip()}")
            print("-"*200)

In [ ]:
print(len(K))
prompts = []
W5 = []
for i, (a, b) in enumerate(K) :
    duplicate = False
    for c in prompts :
        #if jellyfish.jaro_similarity(a, c) >= threshold :
        if c == a :
            duplicate = True
            break
    if duplicate :
        continue
    prompts.append(a)
    out = f"{a.strip()}\nAnswer: {b.strip()}"
    out = out.replace("Therefore, the answer is", "The answer is")
    out = out.replace("Therefore, the final answer is", "The answer is")
    out = out.replace("The answer:", "The answer is")
    out = out.replace("Final answer:", "The answer is")
    out = out.replace("So, the final answer is", "The answer is")
    out = out.replace("So, the answer is", "The answer is")
    for k in [
        "Answer this question based on the passage:",
        "Answer this question:",
    ]:
        out = out.replace(k, "Question:")
    W5.append(out)
    print(out)
    print("-"*200)
print(len(W5))

In [ ]:
cot_qed = Dataset.from_dict(
    {
        "prompt" : [e for e in W5]
    }
)

### cot gsm8k

In [ ]:
for i, example in enumerate(ds):
    if i >= 2000 :
        break
    if "does 3 loads" in example["inputs"] :
        print(i)
        print(example)
        break

In [ ]:
print(example["inputs"])

In [ ]:
triplets = [
    ("Q:", "A:", None),
    ("Question:", "Answer:", None),
    ("[Question]", "[Answer]", None),
    ("Answer the following question.", "Step-by-step reasoning process:", "The answer is"),
    ("Answer this question:", "The answer is", None),
    ("We have a question:", "And an answer:", "So how you got the answer?"),
    ("Reasoning and answer:", "The answer:", "What was the question?\nQuestion:"),
    ("Reasoning & answer:", "The answer:", "Question:"),
    ("Step-by-step reasoning process:", "[Q & A]", "The answer is"),
    ("Consider the Q and A. Q:", "A:", "What is the step-by-step reasoning process?\nStep-by-step reasoning process:"),
    ("Come up with a question and reasoning that would justify this answer:", "Question and rationale:",
    "Step-by-step reasoning process:"),
    (None, "The answer:","Reverse engineering the question:"),
    (None, "Let's think:", None),
    (None, "Let's think step by step.", None)
]
K = []
q = 0
for i, example in enumerate(ds):
    if i >= 1000 :
        break
    if (example["task_name"] != "cot_gsm8k" and example["task_name"] != "cot_gsm8k_ii"):
        continue
    prompt = example['inputs']
    completion = example['targets']
    x_shot = example["template_type"]
    if x_shot == "zs_opt" :
        pass
    else :
        q += 1
        found = False
        z = 0
        while z < 4 :
            for a, b, c in triplets :
                if a is None :
                    if c is None :
                        i2 = prompt.find(b)
                        if i2 == -1 :
                            continue
                        p1 = prompt[0:i2]
                        end = prompt[i2+len(b):].find("\n")
                        if end != -1 :
                            p2 = prompt[i2+len(b):i2+len(b)+end]
                            next = prompt[i2+len(b)+end:].strip()
                        else :
                            p2 = prompt[i2+len(b):]
                            next = ""
                        p3 = "<none>"
                    else :
                        i2 = prompt.find(b)
                        i3 = prompt.find(c)
                        if i2 == -1 or i3 == -1 :
                            continue
                        if i2 >= i3 :
                            continue
                        p1 = prompt[:i2]
                        p2 = prompt[i2+len(b):i3]
                        end = prompt[i3+len(c):].find("\n")
                        if end != -1 :
                            p3 = prompt[i3+len(c):i3+len(c)+end].strip()
                            next = prompt[i3+len(c)+end:].strip()
                        else : 
                            p3 = prompt[i3+len(c):].strip()
                            next = ""
                    found = True
                    prompt = next
                    z += 1
                    break
                else :
                    if prompt.startswith(a) :
                        i1 = 0
                        i2 = prompt.find(b)
                        p1 = prompt[i1+len(a):i2].strip()
                        if c is None :
                            end = prompt[i2+len(b):].find("\n")
                            if end != -1 :
                                p2 = prompt[i2+len(b):i2+len(b)+end]
                                next = prompt[i2+len(b)+end:].strip()
                            else :
                                p2 = prompt[i2+len(b):]
                                next = ""
                            p3 = "<none>"
                        else :
                            i3 = prompt.find(c)
                            p2 = prompt[i2+len(b):i3].strip()
                            end = prompt[i3+len(c):].find("\n")
                            if end != -1 :
                                p3 = prompt[i3+len(c):i3+len(c)+end].strip()
                                next = prompt[i3+len(c)+end:].strip()
                            else : 
                                p3 = prompt[i3+len(c):].strip()
                                next = ""
                        found = True
                        prompt = next
                        z += 1
                        break
                    else :
                        continue
            if len(prompt) == 0 or not found :
                break
            print(f"P1: {p1}\nP2: {p2}\nP3: {p3}")
        print("-"*200)        
        #if not found :
        #    print(f"INPUT\n{prompt}\n\nOUTPUT\n{completion}")
        #    print("-"*200)

### entailment

In [ ]:
couples = [
    ("Premise:", "Hypothesis:"),
    ("Premise:", "Based on this premise, can we conclude that the hypothesis"),
    ("Given the sentence", "is it true that"),
    ("Denny asked: If", "does that mean that"),
    ("Given the sentence", "can we conclude that"),
    ("Student: Can we conclude from", "that"),
    ("Can we conclude from", "that"),
    ("If ", "does that mean that"),
    ("Premise:", "Based on this premise, can we conclude that the hypothesis")
]
tp = [
    "The answer:",
    "the answer is",
    "The answer is",
    "The final answer:",
    "Final answer:"
]
nli_header = "Determine whether the following pairs of sentences embody an entailment relation or not."
nli_end = "A. entailment\nB. no-entailment"
K = []
count = 0
for i, example in enumerate(ds) :
    prompt = example['inputs']
    completion = example['targets']
    if i >= 200000 :
        break
    if "cot_esnli" in  example["task_name"] : # entailement
        if "zs" in example["template_type"] :
            found = False
            for (a, b) in couples :
                if prompt.find(a) == -1 or prompt.find(b) == -1 :
                    continue
                i1 = prompt.find(a)
                subprompt = prompt[i1+len(a):]
                start = subprompt.find('"')
                end = subprompt[start+1:].find('"')
                premise = subprompt[start+1:end+1]
                i2 = prompt.find(b)
                subprompt = prompt[i2+len(b):]
                start = subprompt.find('"')
                end = subprompt[start+1:].find('"')
                hypothesis = subprompt[start+1:end+1]
                found = True
                break
            #K.append((prompt.strip(), completion.strip()))
            if found :
                premise = premise.strip()
                if not premise.endswith(".")  :
                    premise += "."
                hypothesis = hypothesis.strip()
                if not hypothesis.endswith(".") :
                    hypothesis += "."
                out = f"{nli_header}\n\nSentences: {premise} So {hypothesis}\n{nli_end}"
                answer = None
                for k in tp :
                    i1 = completion.find(k)
                    if i1 != -1 :
                        end = completion[i1+len(k):].find(".")
                        if end == -1 :
                            answer = completion[i1+len(k):].strip()
                        else :
                            answer = completion[i1+len(k):i1+len(k)+end].strip()
                if answer is not None :
                    if answer.lower() == "yes" :
                        response = "A"
                    elif answer.lower() == "no" :
                        response = "B"
                    else :
                        response = answer.lower()
                    out += f"\n\nRelation: {response}"
                    if response in ["A", "B"] :
                        K.append(out)
                    #print(out)
                    #print("-"*100)
            else :
                pass
                #print(f"INPUT\n{prompt.strip()}\n\nOUTPUT\n{completion.strip()}")
            #print("-"*200)

In [ ]:
for i, k in enumerate(K):
    if i >= 100 :
        break
    else :
        print(k)
        print("-"*100)

In [ ]:
import jellyfish
prompts = []
W6 = []
threshold=0.95
print(len(K))
for i, a in enumerate(K) :
    duplicate = False
    for c in prompts :
        if c == a :
        #if jellyfish.jaro_similarity(a, c) >= threshold :
            duplicate = True
            break
    if duplicate :
        continue
    prompts.append(a)
    W6.append(a)
    #print(a)
    #print("-"*100)
print(len(W6))

In [ ]:
W6[0]

In [ ]:
cot_nli = Dataset.from_dict(
    {
        "prompt" : [e for e in W6]
    }
)

### creak

In [ ]:
start_blacklist = [
    "Consider the question. Is the following a factual statement?",
    "Question. Yes / no, is the following a verifiable fact?",
    "Q: Claim:", "Consider the question.","The question: Claim:",
    "Q: Yes / no, is the following a verifiable fact?", ""
]
end_blacklist = [

]
start_blacklist_completion = [

]
K = []
for i, example in enumerate(ds) :
    if i >= 20000 :
        break
    if example["task_name"] == "cot_creak_ii" : # yes/no could require to reverse question and answer
        if "zs" in example["template_type"] :
            prompt = example['inputs']
            completion = example['targets']
            for c in start_blacklist :
                if prompt.strip().startswith(c) :
                    prompt = prompt.strip()[len(c):]
                    break
            for c in end_blacklist :
                if prompt.strip().endswith(c):
                    prompt = prompt.strip()[:-len(c)]
                    break
            for c in start_blacklist_completion :
                if completion.strip().startswith(c) :
                    completion = completion.strip()[len(c):]
                    break
            i1 = prompt.find('"')
            if i1 == -1 :
                continue
            i2 = prompt[i1+1:].find('"')
            prompt = prompt[i1+1:i1+1+i2].strip()
            #K.append((prompt.strip(), completion.strip()))
            print(f"INPUT\n{prompt.strip()}\n\nOUTPUT\n{completion.strip()}")
            print("-"*200)

## OpenAssistant

In [ ]:
oasst1 = load_dataset("OpenAssistant/oasst1")

In [ ]:
i = 0
len_ = len(oasst1["train"])
K = []
while i < len_ :
    example = oasst1["train"][i]
    tree_id = example["message_tree_id"]
    role = example["role"]
    j = i+1
    conversation = []
    dico = {}
    dico[role] = example["text"]
    while j < len_ :
        next = oasst1["train"][j]
        if next["message_tree_id"] != tree_id :
            break
        second_role = next["role"]
        dico[second_role] = next["text"]
        if len(dico) == 2 :
            conversation.append(dico)
            dico = {}
        j += 1
    i = j
    K.append(conversation)

In [ ]:
len(K)

In [ ]:
K_clean = []
for i, conv in enumerate(K) :
    prompt = conv[0]['prompter']
    completion = conv[0]['assistant']
    if "as a language model" in completion.lower() :
        continue
    if "sorry" in completion.lower() :
        continue
    print(f"Question: {prompt}\nAnswer: {completion}")
    print("-"*200)
    K_clean.append((prompt, completion))

In [ ]:
for (a, b) in K_clean :
    if 'black' in a :
        out = f"Q: {a}\nA: {b}"
        print(out)
        print("-"*100)

In [ ]:
!pip install fasttext

In [ ]:
import fasttext
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(repo_id="facebook/fasttext-language-identification", filename="model.bin")
model = fasttext.load_model(model_path)
model.predict("Hello, world!")

model.predict("Hello, world!", k=5)


In [ ]:
count = 0
W = []
for prompt, completion in K_clean :
    a, b = model.predict(prompt.strip().split("\n")[0])
    if a[0] == "__label__eng_Latn" and b[0] >= 0.8 :
        out = f"Question: {prompt}\nAnswer: {completion}"
        W.append(out)
        print(out)
        print("-"*200)
        count += 1
print(count)

In [ ]:
oasst_dataset = Dataset.from_dict(
    {
        "prompt" : [e for e in W]
    }
)
oasst_dataset.save_to_disk("oasst_dataset")

## Dolly

In [ ]:
from datasets import load_dataset
dolly = load_dataset("databricks/databricks-dolly-15k", split="train")

In [ ]:
with_context = 0
kept = 0
W = []
for i, example in enumerate(dolly) :
    if len(example["context"].strip()) != 0 :
        with_context += 1
        continue
    if i >= 100000 :
        break
    prompt = example["instruction"]
    completion = example["response"]
    if min(len(prompt), len(completion)) < 100 :
        continue
    out = f"Question: {prompt.strip()}\nAnswer: {completion.strip()}"
    W.append(out)
    print(out)
    print("-"*200)
    kept += 1

In [ ]:
for w in W :
    print(w)
    print("-"*200)

In [ ]:
kept

In [ ]:
dolly_dataset = Dataset.from_dict(
    {
        "prompt" : [e for e in W]
    }
)
dolly_dataset.save_to_disk("dolly_dataset")

## LIMA

In [ ]:
from datasets import load_dataset
lima = load_dataset("GAIR/lima")

In [ ]:
lima

In [ ]:
P = []
for example in lima["train"]:
    conversations = example["conversations"]
    for i in range(0, len(conversations), 2):
        if i >= 2 :
            break
        try :
            prompt = conversations[i]
            completion = conversations[i+1]
            P.append((prompt, completion))
        except :
            pass

In [ ]:
W = [f"Question: {a.strip()}\nAnswer: {b.strip()}" for (a, b) in P]

In [ ]:
for w in W :
    print(w)
    print("-"*200)

In [ ]:
lima_dataset = Dataset.from_dict(
    {
        "prompt" : [e for e in W]
    }
)
lima_dataset.save_to_disk("lima_dataset")

# Merge

In [ ]:
from datasets import concatenate_datasets, load_from_disk
ds = concatenate_datasets(
    [
        load_from_disk(e) for e in [
            "cot_dataset",
            "dolly_dataset",
            "lima_dataset",
            "oasst_dataset",
            "qcm_dataset",
            "summary_dataset"
        ]
    ]
)

In [ ]:
dataset = ds.train_test_split(test_size=0.05)

In [ ]:
dataset.push_to_hub("ArmelRandy/precious")

In [ ]:
for i, e in enumerate(ds):
    if i >= 100 :
        break
    print(e["prompt"])
    print("-"*100)

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
ds = load_dataset("ArmelRandy/precious")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

In [ ]:
n_tokens = 0
for example in ds["train"] :
    n_tokens += len(tokenizer(example["prompt"])["input_ids"])

In [ ]:
print(n_tokens/2048/32)

## Merging

In [ ]:
cot_gsm8k.save_to_disk("cot_gsm8k") # good qcm
cot_aqua.save_to_disk("cot_aqua") # good qcm
cot_ecqa.save_to_disk("cot_ecqa") # good qcm
cot_qed.save_to_disk("cot_qed")
cot_strategyqa.save_to_disk("cot_strategyqa")
cot_nli.save_to_disk("cot_nli") # good

In [ ]:
import numpy as np
rng = np.random.default_rng()

In [ ]:
L = [
    cot_gsm8k,
    cot_aqua,
    cot_ecqa,
    cot_qed,
    cot_strategyqa,
    cot_nli
]
for e in L :
    print(len(e))

In [ ]:
summary_dataset = load_dataset("ArmelRandy/cnndm")

In [ ]:
summary_dataset

In [ ]:
summary = summary_dataset["train"].map(
    lambda x : 
    {
        **x, 
        **{
            "prompt": f"{x['document']}\n\nSummarize the above article in 3 sentences.\n{x['summary']}"
        }
    }
).remove_columns(column_names=summary_dataset["train"].column_names)

In [ ]:
summary = summary.select(rng.choice(len(summary), size=5000, replace=False))

In [ ]:
nli = cot_nli.select(rng.choice(len(cot_nli), size=5000, replace=False))

In [ ]:
cot_strategyqa

In [ ]:
qed = cot_qed.select(rng.choice(len(cot_qed), size=500, replace=False))
#strategy = cot_strategyqa.select(rng.choice(len(cot_strategyqa), size=2000, replace=False))
strategy = cot_strategyqa        

In [ ]:
print(summary, nli, qed, strategy)

In [ ]:
from datasets import load_from_disk
lima = load_from_disk("lima_dataset")
oasst1 = load_from_disk("oasst_dataset")
dolly = load_from_disk("dolly_dataset")
qcm = load_from_disk("flan2021_qcm_dataset")

In [ ]:
precious = concatenate_datasets(
    [
        cot_gsm8k,
        cot_aqua,
        cot_ecqa,
        strategy,
        qed,
        nli,
        summary,
        lima,
        oasst1,
        dolly,
        qcm
    ]
)

In [ ]:
precious = precious.train_test_split(test_size=0.05)

In [ ]:
precious

In [ ]:
precious.push_to_hub("ArmelRandy/more_precious", private=True)